In [2]:
train_input="s3://sagemaker-us-east-2-363557075783/insurance/insurance.csv"

In [16]:
import pandas as pd
df = pd.read_csv(train_input,header=None)

In [3]:
import numpy as np

In [13]:
df.head()

,0,1,2,3,4,5,6
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [17]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [18]:
# spliting the data in to test and train sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5)

In [30]:
train_csv=pd.DataFrame(np.concatenate((X_train,y_train.reshape(-1,1)),axis=1))
train_csv.to_csv('train.csv', header = False, index = False)

In [19]:
test_csv=pd.DataFrame(X_test)
test_csv.to_csv('test.csv', header = False, index = False)

In [32]:
validation_csv=pd.DataFrame(np.concatenate((X_val,y_val.reshape(-1,1)),axis=1))
validation_csv.to_csv('validation.csv', header = False, index = False)

In [35]:
key="XGBoostregressor"

In [41]:
# read the data from csv file and then upload the data to s3 bucket
import os
with open('train.csv','rb') as f:
    # The following code uploads the data into S3 bucket to be accessed later for training
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(f)

# Let's print out the training data location in s3
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://sagemaker-us-east-2-363557075783/insurance/train/XGBoostregressor


In [43]:
# read the data from csv file and then upload the data to s3 bucket
import os
with open('validation.csv','rb') as f:
    # The following code uploads the data into S3 bucket to be accessed later for training
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', key)).upload_fileobj(f)

# Let's print out the training data location in s3
s3_validation_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_validation_data))

uploaded training data location: s3://sagemaker-us-east-2-363557075783/insurance/validation/XGBoostregressor


In [4]:
import sagemaker
import boto3
from sagemaker import Session

# Let's create a Sagemaker session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::363557075783:role/service-role/AmazonSageMaker-ExecutionRole-20210929T104599


In [5]:
bucket = sagemaker_session.default_bucket()
prefix = "insurance"

In [4]:
bucket

'sagemaker-us-east-2-363557075783'

In [6]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'preprocessing.py'
FRAMEWORK_VERSION = "0.23-1"
sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    train_instance_type="ml.m4.xlarge",
    sagemaker_session=sagemaker_session)

sklearn_preprocessor.fit({'train': train_input})

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-11-10 03:20:36 Starting - Starting the training job...
2021-11-10 03:20:38 Starting - Launching requested ML instancesProfilerReport-1636514436: InProgress
...
2021-11-10 03:21:27 Starting - Preparing the instances for training............
2021-11-10 03:23:27 Downloading - Downloading input data
2021-11-10 03:23:27 Training - Downloading the training image...
2021-11-10 03:24:07 Training - Training image download completed. Training in progress..2021-11-10 03:24:08,544 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-11-10 03:24:08,547 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-10 03:24:08,561 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-11-10 03:24:08,860 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-10 03:24:08,875 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-10 03

In [7]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv')
# Preprocess training input
#s3_train_data
transformer.transform('s3://sagemaker-us-east-2-363557075783/insurance/train/XGBoostregressor', content_type='text/csv')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

.................................
2021-11-10 03:30:40,731 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-11-10 03:30:40,734 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-11-10 03:30:40,735 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
    

In [8]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer1 = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv')
# Preprocess training input
#  s3_validation_data
transformer1.transform('s3://sagemaker-us-east-2-363557075783/insurance/validation/XGBoostregressor', content_type='text/csv')
print('Waiting for transform job: ' + transformer1.latest_transform_job.job_name)
transformer1.wait()
preprocessed_validation = transformer1.output_path

................................2021-11-10 03:36:13,056 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-11-10 03:36:13,059 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2021-11-10 03:36:13,060 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      

In [9]:
container = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=boto3.Session().region_name,
    version="1.3-1")

In [10]:
s3_output_key_prefix = "training_output"
s3_output_location = 's3://{}/{}/{}/{}'.format(bucket, prefix, s3_output_key_prefix, 'll_model')

Xgboost_regressor = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count = 1, 
                                       train_instance_type = 'ml.m4.xlarge',
                                       output_path = s3_output_location,
                                       sagemaker_session = sagemaker_session)

#We can tune the hyper-parameters to improve the performance of the model

Xgboost_regressor.set_hyperparameters(max_depth = 6,
                           objective = 'reg:linear',
                           colsample_bytree = 0.2405025950329596,
                           eta=0.3011948580044998,            
                           alpha = 4.25822450396689,
                           num_round = 81
                           )

train_data = sagemaker.session.s3_input(
    preprocessed_train, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')

val_data = sagemaker.session.s3_input(
    preprocessed_validation, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
from sagemaker.serializers import CSVSerializer

Xgboost_regressor.CONTENT_TYPE = 'text/csv'
Xgboost_regressor.serializer = CSVSerializer()
Xgboost_regressor.deserializer = None

data_channels = {'train': train_data,'validation': val_data}
Xgboost_regressor.fit(inputs=data_channels)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-11-10 03:36:47 Starting - Starting the training job...
2021-11-10 03:36:49 Starting - Launching requested ML instancesProfilerReport-1636515407: InProgress
...
2021-11-10 03:37:43 Starting - Preparing the instances for training............
2021-11-10 03:39:47 Downloading - Downloading input data
2021-11-10 03:39:47 Training - Downloading the training image......
2021-11-10 03:40:44 Uploading - Uploading generated training model
2021-11-10 03:40:44 Completed - Training job completed
[2021-11-10 03:40:31.890 ip-10-0-138-176.us-east-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-10:03:40:32:INFO] Imported framework sagemaker_xgboost_container.training
[2021-11-10:03:40:32:INFO] Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
[2021-11-10:03:40:32:INFO] No GPUs detected (normal if no gpus installed)
[2021-11-10:03:40:32:INFO] Running XGBoost Sagemaker in algorithm mode
[2021-11-10:03:40:32:INFO] Determin

In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model(env={"SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT": "text/csv"})
xgboost_model = Xgboost_regressor.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, 
    role=role, 
    models=[
        scikit_learn_inferencee_model, 
        xgboost_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

----------

In [ ]:
def bytes_2_array(x):
    
    # makes entire prediction as string and splits based on ','
    l = str(x).split(',')
    
    # Since the first element contains unwanted characters like (b,',') we remove them
    l[0] = l[0][2:]
    #same-thing as above remove the unwanted last character (')
    l[-1] = l[-1][:-1]
    
    # iterating through the list of strings and converting them into float type
    for i in range(len(l)):
        l[i] = float(l[i])
        
    # converting the list into array
    l = np.array(l).astype('float32')
    
    # reshape one-dimensional array to two-dimensional array
    return l#.reshape(-1,1)

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import numpy as np

payload = '19,female,27.9,0,yes,southwest'
actual_charges = 16884.924
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session,serializer=CSVSerializer()
)

print(bytes_2_array(predictor.predict(payload)))

In [82]:
predictor.predict(X_test)

b'12611.7265625,8754.609375,33730.33984375,17601.87109375,13631.83203125,5119.392578125,10747.9609375,13035.544921875,4897.52392578125,3281.11767578125,5244.5859375,-540.9863891601562,23873.150390625,7232.38525390625,5260.587890625,-1709.5699462890625,3851.597900390625,20907.341796875,4077.118896484375,8947.5732421875,7883.48486328125,5967.8408203125,10389.3642578125,3612.54345703125,5435.45263671875,15514.578125,5911.65283203125,699.138427734375,10705.900390625,7956.42236328125,2952.217041015625,5660.59765625,38420.828125,9004.1455078125,21870.890625,13674.744140625,19069.822265625,6356.96923828125,6549.94921875,10149.6787109375,1918.8697509765625,2838.319580078125,44031.82421875,8518.51171875,30891.005859375,6400.13671875,35448.28125,858.248291015625,25526.486328125,18061.70703125,5398.46630859375,3020.736328125,4395.62158203125,41441.62890625,8040.15087890625,4689.921875,19157.033203125,22098.66796875,8749.072265625,16410.3984375,8905.0546875,18631.18359375,28860.4453125,8628.182617

In [20]:
i=0
predicted_values=[]
result=[]
with open('test.csv', 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        response = predictor.predict(payload)
        charge=bytes_2_array(response)[0]
        predicted_values.append(charge)
        print(row,'------',charge,'------',y_test[i],'\n')
        result.append([row,charge,y_test[i]])
        i+=1
print('done!')

50,male,32.205,0,no,northwest
 ------ 11652.925 ------ 8835.26495 

30,female,28.405,1,no,northwest
 ------ 4232.187 ------ 4527.18295 

37,female,27.74,3,no,northwest
 ------ 6478.019 ------ 7281.5056 

22,female,30.4,0,no,northeast
 ------ 4824.8076 ------ 2741.948 

38,female,30.69,1,no,southeast
 ------ 6797.859 ------ 5976.8311 

57,male,40.945,0,no,northeast
 ------ 12651.37 ------ 11566.30055 

22,female,28.82,0,no,southeast
 ------ 1351.5503 ------ 2156.7518 

42,female,41.325,1,no,northeast
 ------ 8498.025 ------ 7650.77375 

55,female,29.7,2,no,southwest
 ------ 14230.198 ------ 11881.358 

25,male,33.66,4,no,southeast
 ------ 5134.089 ------ 4504.6624 

37,male,30.8,0,no,southwest
 ------ 8211.623 ------ 4646.759 

47,male,32.3,1,no,southwest
 ------ 9292.032 ------ 8062.764 

27,male,33.155,2,no,northwest
 ------ 4781.15 ------ 4058.71245 

35,female,35.815,1,no,northwest
 ------ 5660.5977 ------ 5630.45785 

18,male,34.1,0,no,southeast
 ------ 3626.2673 ------ 1137.011 



In [89]:
predicted_values

[12611.727,
 8754.609,
 33730.34,
 17601.871,
 13631.832,
 5119.3926,
 10747.961,
 13035.545,
 4897.524,
 3281.1177,
 5244.586,
 -540.9864,
 23873.15,
 7232.3853,
 5260.588,
 -1709.57,
 3851.598,
 20907.342,
 4077.119,
 8947.573,
 7883.485,
 5967.841,
 10389.364,
 3612.5435,
 5435.4526,
 15514.578,
 5911.653,
 699.1384,
 10705.9,
 7956.4224,
 2952.217,
 5660.5977,
 38420.83,
 9004.1455,
 21870.89,
 13674.744,
 19069.822,
 6356.969,
 6549.949,
 10149.679,
 1918.8698,
 2838.3196,
 44031.824,
 8518.512,
 30891.006,
 6400.1367,
 35448.28,
 858.2483,
 25526.486,
 18061.707,
 5398.4663,
 3020.7363,
 4395.6216,
 41441.63,
 8040.151,
 4689.922,
 19157.033,
 22098.668,
 8749.072,
 16410.398,
 8905.055,
 18631.184,
 28860.445,
 8628.183,
 -966.967,
 31450.252,
 12668.438,
 4451.1133,
 3818.7307,
 14521.545,
 20056.691,
 6678.9443,
 10578.344,
 17657.873,
 11591.812,
 4908.825,
 17275.049,
 4615.938,
 6042.658,
 13230.5,
 6006.9473,
 5037.5386,
 2009.7944,
 5747.942,
 35127.71,
 21027.385,
 9546.

In [66]:
y_test

array([ 8442.667  ,  3410.324  , 34439.8559 , 17942.106  , 30063.58055,
        6185.3208 ,  6948.7008 ,  8964.06055,  6282.235  ,  3167.45585,
        3046.062  ,  1141.4451 , 17663.1442 ,  2727.3951 ,  6186.127  ,
        1163.4627 ,  3378.91   , 20234.85475, 20277.80751,  5383.536  ,
        7639.41745,  5375.038  ,  3172.018  ,  3392.3652 ,  1712.227  ,
       13393.756  ,  5312.16985,  1842.519  , 10601.63225,  6112.35295,
        1146.7966 ,  5630.45785, 43254.41795,  6781.3542 , 19199.944  ,
       12629.1656 , 19023.26   ,  4827.90495, 20420.60465,  8444.474  ,
        2026.9741 ,  1532.4697 , 47291.055  ,  7243.8136 , 39125.33225,
        3292.52985, 40941.2854 ,  2055.3249 , 21223.6758 , 15612.19335,
        7633.7206 , 21595.38229, 13126.67745, 43813.8661 ,  3659.346  ,
        4673.3922 , 18259.216  , 17361.7661 ,  7626.993  , 11987.1682 ,
       10577.087  , 17560.37975, 24180.9335 ,  9282.4806 ,  1832.094  ,
       40003.33225, 11830.6072 ,  4076.497  ,  4296.2712 , 13390

In [90]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
k = X_test.shape[1]
n = len(X_test)
RMSE = float(format(np.sqrt(mean_squared_error(y_test, predicted_values)),'.3f'))
MSE = mean_squared_error(y_test, predicted_values)
MAE = mean_absolute_error(y_test, predicted_values)
r2 = r2_score(y_test, predicted_values)
adj_r2 = 1-(1-r2)*(n-1)/(n-k-1)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2, '\nAdjusted R2 =', adj_r2) 

RMSE = 5104.97 
MSE = 26060723.468799748 
MAE = 3263.52763412022 
R2 = 0.8060392480974299 
Adjusted R2 = 0.7968757480075447


In [21]:
df=pd.DataFrame(result,columns=['data','predicted','actual'])

In [22]:
df['error%']=df.apply(lambda row: (row.actual - row.predicted)/row.actual*100, axis=1)

In [23]:
df.

,data,predicted,actual,error%
0,"50,male,32.205,0,no,northwest\n",11652.924805,8835.26495,-31.891062
1,"30,female,28.405,1,no,northwest\n",4232.187012,4527.18295,6.516104
2,"37,female,27.74,3,no,northwest\n",6478.019043,7281.50560,11.034621
3,"22,female,30.4,0,no,northeast\n",4824.807617,2741.94800,-75.962769
4,"38,female,30.69,1,no,southeast\n",6797.858887,5976.83110,-13.736841
...,...,...,...,...
129,"47,female,27.83,0,yes,southeast\n",26675.912109,23065.42070,-15.653265
130,"18,female,28.215,0,no,northeast\n",4374.302734,2200.83085,-98.756880
131,"49,female,30.78,1,no,northeast\n",9239.640625,9778.34720,5.509178
132,"58,female,28.215,0,no,northwest\n",12198.149414,12224.35085,0.214338


In [25]:
sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'f42a769a-d4d9-43e8-90c5-79af7b669eee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f42a769a-d4d9-43e8-90c5-79af7b669eee',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 03 Nov 2021 04:36:05 GMT'},
  'RetryAttempts': 0}}

In [24]:
endpoint_name

'inference-pipeline-ep-2021-11-03-03-42-54'